# Tensorflow tutorial

このハンズオンで行うこと
- [Tensorflowの説明](#Tensorflowの説明)
- [計算グラフと実行の練習](#計算グラフと実行の練習)
- [ロジスティック回帰の実装と実行](#ロジスティック回帰の実装と実行)
- [MLPの実装と実行](#MLPの実装と実行)
- [学習済みモデルの保存と学習の再開](#学習済みモデルの保存と学習の再開)
- [TensorBoardの使用](#TensorBoardの使用)
- [精度を上げるためのテクニック](#精度を上げるためのテクニック)
- [Tensorflowのさらなる抽象化](#Tensorflowのさらなる抽象化)

<a name="Tensorflowの説明"></a>
# 0.　Tensorflowとは？

Googleが提供する機械学習用のフレームワーク．
機械学習用のフレームワークは他にもたくさん存在するが，Tensorflowは現在世界で最も使用されているフレームワークであると言われている．

pythonによって書くが、内部はC++やcudaによって書かれている．

'define and run'という形式をとり、まず計算グラフを定義し、それに対してデータを流すという使い方となっている．

以下のコードでフレームワークをインストールすることができる．1.2はTensorflowのバージョン．

In [ ]:
!sudo pip install tensorflow-gpu==1.2

以下のコードで他の必要なライブラリとともに読み込む．

In [ ]:
import tensorflow as tf

from __future__ import print_function

import numpy as np
from sklearn.datasets import fetch_mldata
from sklearn.cross_validation import train_test_split

<a name="計算グラフと実行の練習"></a>
# 1.　計算グラフの構築と実行

計算グラフを構築するためにTensorflow側が用意している型を用いる必要がある.<br>

Tensorflowが用意している種類と使い方は以下のとおり．
1. tf.constant ... ハイパーパラメータなど，実行前から形(shape)の決まった定数に用いる．
2. tf.placeholder ... データの入力など，実行するまでデータのshapeはわからないが変わらないデータを入れるときに用いる(初期化不要)．例えば，データセットの大きさは実行するまでわからない．
3. tf.Variable ... ネットワークの重みなど，学習中に値が変わる最適化対象を入れる(初期化必要)．

## 1.0　計算グラフの実行方法
計算グラフを構築するだけでは，実際に計算は行われない．<br>
計算を実行して値を評価するためには， TensorflowのSessionを作成する必要がある．<br>
例えば，$x$という値の出力が欲しい時は，その値をSessionのrunメソッドに渡してあげる．<br>
具体的には以下のように書けば良い．
```python
with tf.Session() as sess:
    result = sess.run(x)
```

## 1.1　まずはtf.constant(定数)を用いる

In [ ]:
x = tf.constant(1)
y = tf.constant(2)

add_op = tf.add(x, y)
print(x,y)
print(add_op)

ここで表示された結果は定義された計算グラフについての情報で、実際に計算は行われていないことに注意．
- 以下のように計算グラフを実行させて値を確認する．

In [ ]:
with tf.Session() as sess:
    x_, y_, add_op_ = sess.run([x, y, add_op])
    print('x is ',x_)
    print('y is ', y_)
    print('x + y = ',add_op_)

足し算掛け算は以下のようにも書ける．

In [ ]:
x = tf.constant(1)
y = tf.constant(2)

## 足し算掛け算は+,*で書いて良い
add_op = x+y

with tf.Session() as sess:
    x_, y_, add_op_ = sess.run([x, y, add_op])
    print('x is ',x_)
    print('y is ', y_)
    print('x + y = ',add_op_)

## 1.2　tf.placeholderを用いる(データを流す用)

placeholderは初期化不要の変数だが、intかfloatか指定する必要がある．
- tf.float32
- tf.int32

評価対象の変数の計算のために必要なデータの入力はsess.run内のfeed_dict引数内で行うことができる．<br>
feed_dictで渡す変数は一つとは限らないので，辞書型で渡す．

In [ ]:
data = tf.placeholder(tf.int32)
x = tf.constant(5)
op = data*x

with tf.Session() as sess:
    result = sess.run(op, feed_dict={data: 5})
    print('5*5=',result)

    result = sess.run(op, feed_dict={data: 10})
    print('5*10=',result)

## 1.3　tf.Variableを用いる(変数用)

- 実行前に全てのVariableは初期化する必要がある．
    - sess.run(tf.global_variables_initializer())で一度に初期化できる
- Variableへの代入はtf.assignを用いる

In [ ]:
var1 = tf.Variable(0)
const1 = tf.constant(2)

add_op = var1+const1
# Variableへの代入はassignを用いる
var1 = tf.assign(var1, add_op)
print (var1)

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
  
    # var1が毎回更新されている
    print(sess.run([var1]))
    print(sess.run([var1]))
    print(sess.run([var1]))

<a name="ロジスティック回帰の実装と実行"></a>
# 2. ロジスティック回帰

0~9の白黒の手書きの数字のデータセットであるMNISTで10クラス分類をロジスティック回帰によって行う．<br>
目的関数が最小化されるように学習する．
- 目的関数 : softmax cross entropy
    - 入力： $Y=(\boldsymbol{y}_1,\boldsymbol{y}_2,\cdots,\boldsymbol{y}_N)^T\in \mathbb{R}^{N\times K}$, 
$T=(\boldsymbol{t}_1,\boldsymbol{t}_2,\cdots,\boldsymbol{t}_N)^T\in \mathbb{R}^{N\times K}$<br />
$\boldsymbol{y}_n$はsoftmax関数の出力，$\boldsymbol{t}_n$は正解ラベル(1-of-K表現)

    - 出力： $c=\frac{1}{N}\sum_{n=1}^Ncross\_entropy(\boldsymbol{y}_n,\boldsymbol{t}_n) 
    = -\frac{1}{N}\sum_{n=1}^N \sum_{i=1}^K t_{k, i} \log(y_{k, i})\in\mathbb{R}^1$
$$
(\because cross\_entropy(\boldsymbol{y}, \boldsymbol{t})=-\sum_i t_i \log(y_i))
$$

MNISTデータセットの中身は以下の通り．データセットのサイズは$N$とする．
- mnist.data : 画像データ．shapeは$N\times height \times width \times channel$．白黒データなのでチャンネル数$channel=1$. また，画像の高さ$height=28$，幅$width=28$．それぞれのピクセルの値は0~255で表されている．
- mnist.target ： 正解ラベル．shapeは$N\times1$．0~9の値が入っている．

## 2.0　データの準備

In [ ]:
# データのロード
mnist = fetch_mldata('MNIST original', data_home='./data/')

# data : 画像データ， target : 正解ラベル
X, T = mnist.data, mnist.target

# 画像データは0~255の数値のなっているので，0~1の値に変換
X = X / 255.

#　訓練データとテストデータに分ける
X_train, X_test, T_train, T_test = train_test_split(X, T, test_size=0.2)

# データのサイズ
N_train = X_train.shape[0]
N_test = X_test.shape[0]

# ラベルデータをint型に統一し，学習に使いやすいようにone-hot-vectorに変換
T_train = np.eye(10)[T_train.astype("int")]
T_test = np.eye(10)[T_test.astype("int")]

## one-hot-vectorとは？
たとえば$a$が，0~4の整数のみを含むベクトルだとわかっている時に，各行の数字に該当する列の要素のみを1にし，その他を0にする．
$$
\begin{equation*}
a=
\begin{pmatrix}
3\\
1\\
4\\
2\\
0
\end{pmatrix}\to
a\_onehot = 
\begin{pmatrix}
0, 0, 0, 1, 0\\
0, 1, 0, 0, 0\\
0, 0, 0, 0, 1\\
0, 0, 1, 0, 0\\
1, 0, 0, 0, 0
\end{pmatrix}
\end{equation*}
$$
学習における正解ラベルデータは，one-hot-vectorで表されることが多い．

In [ ]:
print ('訓練データのサイズは', N_train)
print ('テストデータのサイズは', N_test)
print ('画像データのshapeは', X_train.shape)
print ('ラベルデータのshapeは', T_train.shape)
print ('ラベルデータの数値の例：')
print (T_train[:10])

## 2.1　ロジスティック回帰クラスの定義
LogisticRegressionのcall関数を実装する．

入力：$\boldsymbol{X}\in\mathbb{R}^{N\times n\_in}$<br>
出力：$\boldsymbol{Y}\in\mathbb{R}^{N\times n\_out}$

<br>
<details>
    <summary>ヒント</summary>
    <div>
    <br>
    - $\boldsymbol{Y}=\boldsymbol{X} * \boldsymbol{W} + \boldsymbol{b}$
    <br>
    - tf.matmul(a, b) : Tensorflowにおける行列の掛け算$a * b$
    </div>
 </details>

In [ ]:
class LogisticRegression:
    def __init__(self, n_in, n_out):
        # n_in : 入力次元数
        # n_out : 出力次元数
        self.W = tf.Variable(tf.zeros([n_in, n_out])) # 重み
        self.b = tf.Variable(tf.zeros(n_out)) # バイアス

    def __call__(self, x):
        ### TODO ###
        y = tf.matmul(x, self.W) + self.b
        ### TODO ###
        return y

## 2.2　グラフの構築

In [ ]:
tf.reset_default_graph()

# パラメータ
# Learning rate (学習率)
lr = 0.7
# epoch数 （学習回数）
n_epoch = 25
# ミニバッチ学習における1バッチのデータ数
batchsize = 100

# 入力
# placeholderを用いると，データのサイズがわからないときにとりあえずNoneとおくことができる．
x = tf.placeholder(tf.float32, [None, 784]) # 28*28次元 
t = tf.placeholder(tf.float32, [None, 10]) # 10クラス

# モデルの定義
# 入力次元数：784，　出力次元数：10
model = LogisticRegression(784, 10)

# y : predictionの結果
y = model(x)

# 目的関数:softmax cross entropy
# 入力：labels->正解ラベル， logits：predictionの結果
# 出力：softmax cross entropyで計算された誤差
xentropy = tf.nn.softmax_cross_entropy_with_logits(labels=t, logits=y)
cost = tf.reduce_mean(xentropy)

# SGD(Stochastic Gradient Descent : 確率的勾配降下法)で目的関数を最小化する
optimizer = tf.train.GradientDescentOptimizer(lr).minimize(cost)

# test用
correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(t, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

## 2.3　グラフの実行

In [ ]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    
    for epoch in range(n_epoch):
        print ('epoch %d | ' % epoch, end="")

        # Training
        sum_loss = 0
        # 訓練データをシャッフルする
        perm = np.random.permutation(N_train)

        for i in range(0, N_train, batchsize):
            # ミニバッチ分のデータを取ってくる
            X_batch = X_train[perm[i:i+batchsize]]
            t_batch = T_train[perm[i:i+batchsize]]

            _, loss = sess.run([optimizer, cost], feed_dict={x:X_batch, t:t_batch})
            sum_loss += loss * X_batch.shape[0]

        loss = sum_loss / N_train
        print('Train loss %.3f | ' %(loss), end="")

        # Test model
        print ("Accuracy: %.3f"%(accuracy.eval(feed_dict={x: X_test, t: T_test})))

<a name="MLPの実装と実行"></a>

# 3.　MLP

多層のニューラルネットワークを学習させる．<br>
層を深くすることで表現力が上がり、性能の向上が期待できる．<br>
データは先ほどと同じMNISTを用いて比較する．

データを読み込みたい場合は，2.0データの準備参照．

## 3.0　MLPクラスの定義
以下のような要件のネットワークを構築する．
```
    入力 : x　
-> Fully connected layer 1 (input : x, outputの次元数 : 256, 活性化関数 : relu関数)
-> Fully connected layer 2 (input : layer1の出力， outputの次元数 : 256, 活性化関数 : relu関数)
-> Fully connected layer 3 (input : layer2の出力， outputの次元数 : 10)
-> 出力 : out
```

<details>
    <summary>ヒント</summary>
    <div><br>
    - TensorflowでFully connected layerはtf.layers.dense (inputs, units, activation=None)で呼ぶことができる．
    <br>
    - inputs : 入力データ
    <br>
    - units :  outputの次元数
    <br>
    - activation : 活性化関数の種類（デフォルトでは無し）
    <br>
    - relu関数はTensorflowでtf.nn.reluと表される．
    </div>
</details>

In [ ]:
def MLP(x):
    ### TODO
    layer_1 = tf.layers.dense(x, 256, activation=tf.nn.relu)
    layer_2 = tf.layers.dense(layer_1, 256, activation=tf.nn.relu)
    out = tf.layers.dense(layer_2, 10)
    ### TODO
    return out

## 3.1　グラフの構築

In [ ]:
tf.reset_default_graph()

# パラメータ
# Learning rate (学習率)
lr = 0.1
# epoch数 （学習回数）
n_epoch = 25
# ミニバッチ学習における1バッチのデータ数
batchsize = 100

# 入力
# placeholderを用いると，データのサイズがわからないときにとりあえずNoneとおくことができる．
x = tf.placeholder(tf.float32, [None, 784]) # 28*28次元 
t = tf.placeholder(tf.float32, [None, 10]) # 10クラス

# MLPクラスのモデルを用いてpredictionを行う
y = MLP(x)

# 目的関数:softmax cross entropy
# 入力：labels->正解ラベル， logits：predictionの結果
# 出力：softmax cross entropyで計算された誤差
xentropy = tf.nn.softmax_cross_entropy_with_logits(labels=t, logits=y)
cost = tf.reduce_mean(xentropy)

# SGD(Stochastic Gradient Descent : 確率的勾配降下法)で目的関数を最小化する
optimizer = tf.train.GradientDescentOptimizer(lr).minimize(cost)

# test用
correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(t, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

## 3.2　グラフの実行

In [ ]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    
    for epoch in range(n_epoch):
        print ('epoch %d | ' % epoch, end="")

        # Training
        sum_loss = 0
        # 訓練データをシャッフルする
        perm = np.random.permutation(N_train)

        for i in range(0, N_train, batchsize):
            # ミニバッチ分のデータを取ってくる
            X_batch = X_train[perm[i:i+batchsize]]
            t_batch = T_train[perm[i:i+batchsize]]

            _, loss = sess.run([optimizer, cost], feed_dict={x:X_batch, t:t_batch})
            sum_loss += loss * X_batch.shape[0]

        loss = sum_loss / N_train
        print('Train loss %.5f | ' %(loss), end="")

        # Test model
        print ("Test Accuracy: %.3f"%(accuracy.eval(feed_dict={x: X_test, t: T_test})))

<a name="学習済みモデルの保存と学習の再開"></a>
# 4.　モデルの保存と再利用

- tf.train.Saver()を用いる

ここでは，先ほどグラフを構築したMLPをそのまま用いる．

## 4.0　モデルを保存するディレクトリを作成する

In [ ]:
import os

# モデルの保存先ディレクトリのpath
model_path = "./model/"

# もし該当のディレクトリが存在しなかったらディレクトリを作成する
if not os.path.exists(model_path):
    os.mkdir(model_path)

## 4.1　モデルを保存する

In [ ]:
# tf.train.Saverのコンストラクタ
saver = tf.train.Saver()

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    
    for epoch in range(5):
        print ('epoch %d | ' % epoch, end="")

        # Training
        sum_loss = 0
        # 訓練データをシャッフルする
        perm = np.random.permutation(N_train)

        for i in range(0, N_train, batchsize):
            # ミニバッチ分のデータを取ってくる
            X_batch = X_train[perm[i:i+batchsize]]
            t_batch = T_train[perm[i:i+batchsize]]

            _, loss = sess.run([optimizer, cost], feed_dict={x:X_batch, t:t_batch})
            sum_loss += loss * X_batch.shape[0]

        loss = sum_loss / N_train
        print('Train loss %.5f | ' %(loss), end="")

        # Test model
        print ("Test Accuracy: %.3f"%(accuracy.eval(feed_dict={x: X_test, t: T_test})))
        
        ###　モデルの保存
        saver.save(sess, model_path)

## 4.2　上で保存したモデルを読み込み，学習を再開する

In [ ]:
with tf.Session() as sess:
    #　model_pathで指定されたモデルを読み込む
    saver.restore(sess, model_path)
    
    for epoch in range(5):
        print ('epoch %d | ' % epoch, end="")
        
        # Training
        sum_loss = 0
        # 訓練データをシャッフルする
        perm = np.random.permutation(N_train)

        for i in range(0, N_train, batchsize):
            # ミニバッチ分のデータを取ってくる
            X_batch = X_train[perm[i:i+batchsize]]
            t_batch = T_train[perm[i:i+batchsize]]

            _, loss = sess.run([optimizer, cost], feed_dict={x:X_batch, t:t_batch})
            sum_loss += loss * X_batch.shape[0]

        loss = sum_loss / N_train
        print('Train loss %.5f | ' %(loss), end="")

        # Test model
        print ("Test Accuracy: %.3f"%(accuracy.eval(feed_dict={x: X_test, t: T_test})))

<a name="TensorBoardの使用"></a>
# 5. TensorBoardを用いて学習経過を可視化する

TensorBoardとは，Tensorflowが提供する可視化ツール．<br>
学習経過やグラフ構造をブラウザ上で動的に確認できる．

## 5.0　logを保存する用のディレクトリを作成する

In [ ]:
import os
import shutil

# logの保存先ディレクトリのpath
logs_path = './log/'

# もし該当のディレクトリが存在したら削除する
if os.path.exists(logs_path):
    shutil.rmtree(logs_path)
    
# 保存先ディレクトリを作成する
os.mkdir(logs_path)

## 5.1　グラフを構築する

In [ ]:
tf.reset_default_graph()

# パラメータ
# Learning rate (学習率)
lr = 0.001
# epoch数 （学習回数）
n_epoch = 25
# ミニバッチ学習における1バッチのデータ数
batchsize = 100

# 入力
# placeholderを用いると，データのサイズがわからないときにとりあえずNoneとおくことができる．
x = tf.placeholder(tf.float32, [None, 784]) # 28*28次元 
t = tf.placeholder(tf.float32, [None, 10]) # 10クラス

# MLPクラスのモデルを用いてpredictionを行う
y = MLP(x)

# 目的関数:softmax cross entropy
# 入力：labels->正解ラベル， logits：predictionの結果
# 出力：softmax cross entropyで計算された誤差
xentropy = tf.nn.softmax_cross_entropy_with_logits(labels=t, logits=y)
cost = tf.reduce_mean(xentropy)

# SGD(Stochastic Gradient Descent : 確率的勾配降下法)で目的関数を最小化する
optimizer = tf.train.GradientDescentOptimizer(lr).minimize(cost)

# test用
correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(t, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

# 可視化対象に誤差を設定する
tf.summary.scalar("train_loss", cost)
# 可視化対象に正解率を設定する
tf.summary.scalar("train_accuracy", accuracy)
# 全ての可視化対象を統合して一つの操作にまとめる
merged_summary_op = tf.summary.merge_all()

## 5.2　グラフを実行する

In [ ]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    summary_writer = tf.summary.FileWriter(logs_path, graph=tf.get_default_graph())
    
    for epoch in range(n_epoch):
        print ('epoch %d | ' % epoch, end="")

        # Training
        sum_loss = 0
        # 訓練データをシャッフルする
        perm = np.random.permutation(N_train)

        for i in range(0, N_train, batchsize):
            # ミニバッチ分のデータを取ってくる
            X_batch = X_train[perm[i:i+batchsize]]
            t_batch = T_train[perm[i:i+batchsize]]

            _, loss, summary = sess.run([optimizer, cost, merged_summary_op], feed_dict={x:X_batch, t:t_batch})
            summary_writer.add_summary(summary, epoch * N_train + len(perm[:i+batchsize]))
            sum_loss += loss * X_batch.shape[0]

        loss = sum_loss / N_train
        print('Train loss %.5f | ' %(loss), end="")

        # Test model
        print ("Test Accuracy: %.3f"%(accuracy.eval(feed_dict={x: X_test, t: T_test})))

    summary_writer.close()

## 5.3　TensorBoardを立ち上げる
上記の学習を始めたら、terminal上で以下を実行
- tensorboard --logdir ./log/
(logのディレクトリを指定) 
- ブラウザ上でnotebookのhttp://{サーバーのアドレス}:6006/ にアクセスする．
- サーバーのアドレスとは，現在用いているjupyter notebookのURLのうち，"ec2-....compute.amazonaws.com"で表される形式の文字列．

<a name='精度を上げるためのテクニック'></a>
# 6. 精度を上げるためのテクニック

初期設定では，ロジスティック回帰，MLPともに98%前後のaccuracyになったのではないかと思います．

こうした学習においては，以下のような重みの初期値，Dropout，data augmentationなどのテクニックを用いて新たなモデルを作成することで，さらに性能が向上する可能性があります．

モデルを工夫して，よりよい性能を発揮するネットワークを構築してみましょう．


## 6.0 パラメータの初期化

### Heの初期化
```python
he_init = tf.variance_scaling_initializer()
hidden = tf.layers.dense(X, n_hidden, activation=tf.nn.relu, kernel_initializer=he_init, name="hidden")
```

### Xavierの初期化
```python
xavier_init = tf.glorot_uniform_initializer()
hidden = tf.layers.dense(X, n_hidden, activation=tf.nn.sigmoid, kernel_initializer=xavier_init, name="hidden")
```

## 6.1 バッチ正規化

```python
hidden = tf.layers.dense(X, n_hidden, name="hidden")
bn = tf.layers.batch_normalization(hidden, training=training, momentum=0.9)
bn_act = tf.nn.relu(bn)
```


## 6.2 最適化手法


### Momentum
```python
optimizer = tf.train.MomentumOptimizer(learning_rate=learning_rate, momentum=0.9)
```

### AdaGrad
```python
optimizer = tf.train.AdagradOptimizer(learning_rate=learning_rate)
```

### Adam
```python
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
```


## 6.3 過学習防ぐための正則化

### Weight Decay
loss関数にネットワークの重みを加えることで，パラメータが大きくなりすぎることを防ぐ．
```python
L2_sqr = tf.nn.l2_loss(W)
lambda_2 = 0.01
loss = cross_entropy + lambda_2 * L2_sqr
```

### Dropout
```python
dropout_rate = 0.5
hidden = tf.layers.dense(X, n_hidden, activation=tf.nn.relu, name="hidden")
hidden_drop = tf.layers.dropout(hidden, dropout_rate, training=training)
```


## 6.4 Data Augmentation（データ拡張）

- 拡大縮小
- 回転
- 反転
- ノイズ追加
- クロップ
などなど

<a name='Tensorflowのさらなる抽象化'></a>
# < Tensorflow 応用編 >

Tensorflowには学習コードをさらに抽象化する<strong>Estimator</strong>というものがある．<br>
Estimatorを用いるとEstimator.train()を呼ぶだけでepoch数だけfor文を書くなどの手間が省けるため，コードの可読性が上がり，<br>
実用でも広く使われている．

## Estimatorの使い方
Estimatorの使用方法について大まかなコードの流れの例を説明したものが以下のサンプルコードである．
```python
estimator = tf.estimator.Estimator(model_fn)
estimator.train(train_input_fn)
result = estimator.evaluate(test_input_fn)
```
model_fnにモデルの内容（ネットワーク構造，ロス関数，optimizerの設定）を書き，Estimatorに渡す．<br>
Estimatorは訓練用のデータの情報が入っているtrain_input_fnを引数としてtrainメソッドを呼ぶことで訓練を行う．<br>
さらに，テスト用のデータの情報が入っているtest_input_fnを引数としてevaluateメソッドを呼ぶことでテストを行い，学習データになかったデータに対する汎用性を評価する．

In [ ]:
# Estimatorでの実装時に用いられる関数はone-hot-vectorではなくクラスラベルをそのまま入力することが多い
t_train = np.argmax(T_train, axis=1)
t_test = np.argmax(T_test, axis=1)

In [ ]:
def my_model(features, labels, mode, params):
    # モデルを定義する
    net = features['x']
    for units in params['hidden_units']:
        # 入力がnetで出力がunits個，活性化関数がrelu関数である全結合層
        net = tf.layers.dense(net, units=units, activation=tf.nn.relu)

    # netを入力としてクラス分のunit数に出力することでlogitsを計算する
    logits = tf.layers.dense(net, params['n_classes'], activation=None)
    
    # ロスの定義
    loss = tf.losses.sparse_softmax_cross_entropy(labels=labels, logits=logits)

    # 推定結果を計算する
    predicted_classes = tf.argmax(logits, 1)
    
    # 正解率の計算方法を指定する
    accuracy = tf.metrics.accuracy(labels=labels, predictions=predicted_classes)
    metrics = {'accuracy': accuracy}
  
    # evaluationモードのとき
    if mode == tf.estimator.ModeKeys.EVAL:
        return tf.estimator.EstimatorSpec(mode, loss=loss, eval_metric_ops=metrics)

    # lossを最小化するオプティマイザを定義する
    optimizer = tf.train.AdagradOptimizer(learning_rate=0.1)
    train_op = optimizer.minimize(loss, global_step=tf.train.get_global_step())
     
    # trainモードのとき
    return tf.estimator.EstimatorSpec(mode, loss=loss, train_op=train_op)

In [ ]:
def main():
    batch_size = 100

    # 訓練データ用
    train_input_fn = tf.estimator.inputs.numpy_input_fn(
        x={"x": X_train},
        y=t_train,
        batch_size=batch_size,
        num_epochs=5,
        shuffle=True)
    
    # テストデータ用
    eval_input_fn = tf.estimator.inputs.numpy_input_fn(
        x={"x": X_test},
        y=t_test,
        num_epochs=1,
        shuffle=False)

    # model functionを用いてestimatorを定義する
    estimator = tf.estimator.Estimator(
        model_fn=my_model,
        params={
            'hidden_units': [256, 256], #  256ノード持つ2つの隠れ層
            'n_classes': 10, # モデルは結果を3つのクラスから選ぶ
        })
     
    # モデルを訓練
    estimator.train(input_fn=train_input_fn)
    
    # モデルの評価
    eval_result = estimator.evaluate(input_fn=eval_input_fn)
    
    print('\nTest set accuracy: {accuracy:0.3f}\n'.format(**eval_result))

In [ ]:
main()